<a href="https://colab.research.google.com/github/thomasshin/NLP_Study/blob/main/Andrej_Karpathy_Neural_Network_Zero_to_Hero/Exercises_from_lecture_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [ ]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos, exp

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [ ]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  h = 0.00000001
  dfda = (f(a+h, b, c) - f(a, b, c)) / h
  dfdb = (f(a, b+h, c) - f(a, b, c)) / h
  dfdc = (f(a, b, c+h) - f(a, b, c)) / h
  return [dfda, dfdb, dfdc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553380251014
OK for dim 1: expected 10.25699027111255, yours returns 10.256990368162633
OK for dim 2: expected 0.0625, yours returns 0.0624999607623522


In [ ]:
class Value:
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self.label = label
    self._prev = set(_children)
    self._op = _op

  def __repr__(self):
    return "Value(data={})".format(self.data)

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other) #for Value + 2
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
      self.grad += other * (self.data ** (other-1)) * out.grad
    out._backward = _backward

    return out

  def __rmul__(self, other): #other * self
    return self * other

  def __radd__(self, other):
    return self + other

  def __truediv__(self, other): #self / other
    return self * other**-1

  def __neg__(self):
    return self * -1

  def __sub__(self, other):
    return self + (-other)

  def tanh(self):
    x = self.data
    t = (exp(2*x)-1) / (exp(2*x)+1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward

    return out

  def sin(self):
    x = self.data
    out = Value(sin(x), (self, ), 'sin')

    def _backward():
      self.grad += (cos(x)) * out.grad
    out._backward = _backward

    return out

  def exp(self):
    x = self.data
    out = Value(exp(x), (self,), 'exp')

    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward

    return out

  def backward(self):

    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v) #v will not be appended unless its children are all appended first
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [ ]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

#f(a,b,c) = -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

# -----------
a = Value(2.0)
b = Value(3.0)
c = Value(4.0)
y = -1.0*a**3 + ((3*b).sin()) - (1.0*c**-1) + (b**2.5) - (a**0.5)
y.backward()
numerical_grad = [a.grad, b.grad, c.grad] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs (numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [ ]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
a = 2
b = 3
c = 4
h = 0.00000001
dfda_ = (f(a+h, b, c) - f(a-h, b, c)) / (2*h)
dfdb_ = (f(a, b+h, c) - f(a, b-h, c)) / (2*h)
dfdc_ = (f(a, b, c+h) - f(a, b, c-h)) / (2*h)
numerical_grad2 = [dfda_, dfdb_, dfdc_] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553291433172
OK for dim 1: expected 10.25699027111255, yours returns 10.256990368162633
OK for dim 2: expected 0.0625, yours returns 0.0624999607623522


## section 2: support for softmax

In [ ]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  # ------

  def exp(self):
    x = self.data
    out = Value(exp(x), (self,), 'exp')

    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float))
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
      self.grad += other * (self.data ** (other-1)) * out.grad
    out._backward = _backward

    return out

  def __radd__(self, other):
    return self + other

  def __truediv__(self, other): #self / other
    out = Value(self.data * other.data**-1, (self, other), '/')

    def _backward():
      self.grad += other.data**-1 * out.grad
      other.grad += -self.data * other.data**-2 * out.grad
    out._backward = _backward

    return out

  def __neg__(self):
    return self *-1

  def log(self): #ln
    out = Value(log(self.data), (self,), 'log')

    def _backward():
      self.grad += (1/self.data) * out.grad
    out._backward = _backward

    return out

  def __rmul__(self, other):
    return self * other

  # ------

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [ ]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function


def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535044
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625318
OK for dim 2: expected 0.005653302662216329, yours returns 0.0056533026622163285
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [ ]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
import torch.nn.functional as F

In [ ]:
logits = torch.Tensor([0.0, 3.0, -2.0, 1.0]).double() ; logits.requires_grad = True
probs = F.softmax(logits, dim=0)
loss = -torch.log(probs[3])
loss.backward()
logits.grad

tensor([ 0.0418,  0.8390,  0.0057, -0.8865], dtype=torch.float64)